In [3]:
import numpy as np, scipy, matplotlib
import matplotlib.pyplot as pl
from scipy.optimize import curve_fit
import matplotlib.ticker as ticker
from scipy.signal import detrend
import scipy.constants as const
%pylab

g031_freq, g031_temp = np.loadtxt("G031.727.dat", unpack = 1)

#Get the stupid frequencies in ascending order
g031_freq = g031_freq[:: -1] * 1e6
g031_temp = g031_temp[:: -1]

#Trim off the first 500 samples corresponding to bad baseline
g031_freq = g031_freq[300:]
g031_temp = g031_temp[300:]

#pl.plot(g031_freq, g031_temp)

#Script to remove baseline
#Use polyfit instead of curve_fit as don't know the function

coeff = np.polyfit(g031_freq, g031_temp, 3)
baseline = np.poly1d(coeff)

g031_temp_corr = g031_temp - baseline(g031_freq)

#pl.plot(g031_freq, g031_temp_corr)

#Plot the raw data and corrected data

fig, (ax0, ax1) = plt.subplots(nrows = 2, ncols = 1, sharex = True, sharey = True)

ax0.plot(g031_freq, g031_temp, 'r')
ax1.plot(g031_freq, g031_temp_corr, 'r')

#ax0.set_ylabel("Antenna Temperature (K)", fontsize = 22)
ax1.set_xlabel("Frequency (Hz)", fontsize = 22)

fig.text(0.00, 0.5, 'Antenna Temperature (K)', va='center', rotation='vertical', fontsize = 20)

pl.savefig("g031_raw_corr.pdf")
pl.show()

#Smooth the data, cause I can't see the line in this:

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

#pl.plot(range(len(g031_freq)), g031_temp_corr)

#Fit that weak line towards the end of the spectrum:

def gaussian(x, a, b, c):
    
    return a * np.exp(-(x - b) ** 2 / (2 * c ** 2))

popt, pconv = curve_fit(gaussian, g031_freq[3100: 3300], g031_temp_corr[3100: 3300], p0 = (0.1, 9815e6, 1e6), maxfev = 2000)

print(popt)
print(np.diag(pconv) ** 2)

hpbw = popt[2] * 2.355 

fig, ax = pl.subplots()

ax.plot(g031_freq[3000: 3500], g031_temp[3000: 3500], color = 'r', label = 'data')
ax.plot(g031_freq[3000: 3500], gaussian(g031_freq[3000: 3500], *popt), color = 'g', label = 'fit')

ax.set_xlabel("Frequency (MHz)", fontsize = 22)
ax.set_ylabel("Antenna Temperature (K)", fontsize = 22)

fig.tight_layout()
pl.legend(loc = 'best', frameon = False, fontsize = 18)
pl.savefig("g031_best_fit.pdf")

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib
[2.35659383e-03 9.81327447e+09 2.60862123e+05]
[1.11408487e-15 2.94682187e+17 2.96288046e+17]


In [25]:
#Calculate electron temperature and line brightness temperature and EM
#Look at equations 7.35 and 7.97 in ERA

T_e = const.m_p * (const.c ** 2 / (8 * np.log(2) * const.k) ) * (hpbw / popt[1]) ** 2 

print T_e

9362.050518672044


In [27]:
#Calculate the line brightness and EM here (Eq 7.97 in ERA):

A_e = 5262.9792822
omega = 1.87605047039e-07  #These values taken from continuum_analysis notebook

T_b = gaussian(popt[1], *popt) * const.c ** 2 / (A_e * omega * popt[1] ** 2)

print T_b * 1e3

2.307929362135442


In [28]:
#EM stuff here:

EM = T_b * (T_e ** 1.5) * (hpbw / 1e3) / (1.92e3)

print EM

737.8307716243816


In [30]:
#Estimate source size here using ERA eq. 7.95 and assuming all EM comes from source, none
#from electrons along LOS:

n_e = 1e3 #Need to find references for this. Source is Cas A

L = EM / (n_e ** 2)   #in pc

#convert L to lyr:
#L = L * 3.26156

print "Source size:", L * 1e3, "pc"

Source size: 0.7378307716243816 pc
